In [1]:
########################start##############################
import pandas as pd
df=pd.read_csv('./datas/Data_MergeNew.csv',index_col=0)
df.index=pd.to_datetime(df.date,format='%Y-%m-%d') 
df.drop('date',axis=1,inplace=True)
df_train=df.loc['2020-01-07':'2021-04-02'] 
df_test=df.loc['2021-04-02':'2021-04-23']

name=['cityin','cityto','incity','city']
name1=['cityin','cityto','incity']
x_train=df_train.drop(name,axis=1).values  
y_train=df_train[name1].values
x_test=df_test.drop(name,axis=1).values  
y_test=df_test[name1].values
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

#####################标准化##############################
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train) 
x_test = ss.transform(x_test)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,4,6,8,20,21,23,25,34,36,38,40,56,59,60,63,64,65,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(53370, 65) (17755, 65) (53370, 3) (17755, 3)


In [2]:
import csv
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("white")
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)            
        print("---  new folder...  ---")
        print("---  OK  ---")
    else:
        print("---  There is this folder!  ---")
mkdir("./results_merge/")        
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_squared_log_error

def calculate(y_true, y_predict, n, p):
    y_true = y_true
    y_predict = y_predict
    mse = mean_squared_error(y_true, y_predict)
    rmse = np.sqrt(mean_squared_error(y_true, y_predict))
    mae = mean_absolute_error(y_true, y_predict)
    r2 = r2_score(y_true, y_predict)
    mad = median_absolute_error(y_true, y_predict)
    mape = np.mean(np.abs((y_true - y_predict) / y_true)) * 100
    r2_adjusted = 1-((1-r2)*(n-1))/(n-p-1)
    # print('MSE: ', mse)
    # print('RMSE: ', rmse)
    # print('MAE: ', mae)
    # print('R2: ', r2)
    # print('MAD:', mad)
    # print('MAPE:', mape)
    # print('R2_Adjusted: ',r2_adjusted)
    return mse,rmse,mae,r2,mad,mape,r2_adjusted
# 将参数和评估结果写入文件
def write_csv_result(path_1,path_2,all_metrics,all_parameter):
    with open(path_1,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_metrics)
    with open(path_2,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_parameter)


---  new folder...  ---
---  OK  ---


In [6]:
from itertools import combinations
from sklearn.neural_network import MLPRegressor

from sklearn.ensemble import BaggingRegressor
import csv
import os
import numpy as np
from sklearn.multioutput import RegressorChain
def adjust_mlp(train_x,test_x,train_y,test_y,name):
    path_a = "./results_merge/" + "Mlp_" + name + "_" + "assess.csv"
    path_p = "./results_merge/" + "Mlp_" + name + "_" + "parameter.csv"
    path_b = "./results_merge/" + "Mlp_" + name + "_" + "best.csv"
    model_path = "./results_merge/" + "Mlp_" + name + "_" + "best.m"
    all_assessed_values = []
    all_parameter = []
    max_iter = [5000,10000,15000,20000]
    tol = [1e-3,2e-3,1e-4,1e-2]
    learning_rate_init = [1e-2,1e-3,1e-4]
    hidden_layer_sizes = list(combinations([64,32,16,8,4], 3))
    all_nb = len(max_iter) * len(tol) * len(learning_rate_init) * len(hidden_layer_sizes)
    num = 1
    # 用于重启训练模型，提高效率，不重复跑相同的实验
    if(os.path.exists(path_a)):
        data = pd.read_csv(path_a)
        if(data.shape[0]>1):
            try:
                nums = int(data.values[-1,0])
            except:
                nums=0
        else:
            nums = 0
    else:
        nums = 0
        col_a = ['num','mse','rmse','mae','r2','mad','mape','r2_adjusted']
        col_p = ['num','max_iter','tol','learning_rate_init','hidden_layer_sizes']
        write_csv_result(path_a,path_p,col_a,col_p)
    # 用于保存最好的模型
    if(os.path.exists(path_b)):
        # best_results = pd.read_csv(path_b)
        # if(best_results.shape[0]>1):
        #     best_result = best_results["rmse"].values[-1]
        # else:
        best_result = 10*10**30
    else:
        with open(path_b,"a",encoding="utf-8",newline="")as f:
            f = csv.writer(f)
            f.writerow(['num','mse','rmse','mae','r2','mad','mape','r2_adjusted'])
        best_result = 10*10**30

    for m in max_iter:
        for t in tol:
            for l in learning_rate_init:
                for hd in hidden_layer_sizes:
                    if(nums>=num):
                        num = num+1
                    else:
                        print("start....{}/{}".format(num,all_nb))
                        mlp = MLPRegressor(hidden_layer_sizes=hd, activation="relu",
                                         solver='adam', alpha=0.0001,
                                         batch_size='auto', learning_rate="constant",
                                         learning_rate_init=l,
                                         power_t=0.5, max_iter=m,tol=t)
                        wrapper = RegressorChain(mlp)
                        wrapper.fit(train_x,train_y)
                        pred_test = wrapper.predict(test_x)

                        y_test=test_y       
                        y_test1=[]
                        for i in y_test:
                            y_test1.append(i[0])
                        pred_test1=[]
                        for i in pred_test:
                            pred_test1.append(i[0])
                        y_test2=[]
                        for i in y_test:
                            y_test2.append(i[1])
                        pred_test2=[]
                        for i in pred_test:
                            pred_test2.append(i[1])
                        y_test3=[]
                        for i in y_test:
                            y_test3.append(i[2])
                        pred_test3=[]
                        for i in pred_test:
                            pred_test3.append(i[2])
                        pred_test1 = np.array(pred_test1).reshape(-1,1)
                        y_test1 = np.array(y_test1).reshape(-1,1)
                        sample_n = pred_test1.shape[0]
                        feature_n = x_test.shape[1]
                        mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)


                        pred_test2 = np.array(pred_test2).reshape(-1,1)
                        y_test2 = np.array(y_test2).reshape(-1,1)
                        sample_n = pred_test2.shape[0]
                        feature_n = x_test.shape[1]
                        mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)


                        pred_test3 = np.array(pred_test3).reshape(-1,1)
                        y_test3 = np.array(y_test3).reshape(-1,1)
                        sample_n = pred_test3.shape[0]
                        feature_n = x_test.shape[1]
                        mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)


                        pred_test4 = pred_test.reshape(-1,1)
                        y_test4 = y_test.reshape(-1,1)
                        sample_n = pred_test4.shape[0]
                        feature_n = x_test.shape[1]
                        mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3
                        all_m = [num,mse,rmse,mae,r2,mad,mape,r2_adjusted]
                        all_p = [num,m,t,l,hd]
                        # print(all_m)
                        write_csv_result(path_a,path_p,all_m,all_p)  

                        if(rmse < best_result):
                            joblib.dump(filename=model_path,value=mlp,compress=True)
                            # joblib.dump(model,model_path)
                            with open(path_b,"w",encoding="utf-8",newline="")as f:
                                f = csv.writer(f)
                                f.writerow([num,mse,rmse,mae,r2,mad,mape,r2_adjusted])
                                f.writerow([num,mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1])
                                f.writerow([num,mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2])
                                f.writerow([num,mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3])
                            best_result = rmse
                            print("current best result, mse:{},mae:{},rmse:{},mad:{},r2:{},mape:{},r2 adjusted:{}".format(mse,mad,rmse,mad,r2,mape,r2_adjusted))
                        print("end....",num)
                        num = num+1
                        print("--------------------------------")  
                    
name='Migration'
adjust_mlp(x_train,x_test,y_train,y_test,name)

start....1/480


D:\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 